# Imports

In [25]:
'''
Imports for Parsing
'''
import os
import tarfile
import xml.dom.minidom
import pandas as pd
from xml.etree import cElementTree as ET
import warnings
warnings.filterwarnings('ignore')

'''
Imports for Algorithm
'''
import nltk; 
#nltk.download('stopwords')
from nltk.corpus import stopwords

import re
import numpy as np
import pandas as pd
import scipy as sp
from pprint import pprint

# Gensim LDA
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
from spacy.cli import download

# causalty measure
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

# Parsing Data

In [26]:
# RETURN: Array of tuples containing date and doc content
# [(date, document_content)]
def get_timestamped_docs():
    docs = [] #(date, document_data)
    for month in os.listdir("./documents"):
        for day in os.listdir("./documents/"+month):
            for doc in os.listdir("./documents/"+month+"/"+day):
                tree = ET.parse("./documents/"+month+"/"+day+"/"+doc)
                root = tree.getroot()
                for page in root.findall('body'):
                    for page1 in page.findall('body.content'):
                        for page2 in page1.findall('block'):
                            for page3 in page2.findall('p'):
                                if page3.text != None:
                                    docs.append((month+"/"+day+"/00", page3.text)) 
            break
        break
    return docs
        
# RETURN: documents that contain "Gore" or "Bush" 
# documents: [String], timestamps: [String] 
def filter_docs(docs):
    D = [] # document contents 
    T = [] # corresponding timestamps for documents
    for i, doc in enumerate(docs):
        gore = 0
        bush = 0
        if "Gore" in doc[1]:
            gore += 1
        if "Bush" in doc[1]:
            bush += 1
        if gore > 0 or bush > 0:
            T.append(doc[0])
            D.append(doc[1])
    return (D, T)

In [27]:
docs = get_timestamped_docs()
D, T = filter_docs(docs) # returns documents and their timestamps

# Constants

In [28]:
NUM_TOPICS = 10

# LDA Initial Pass (Generate Topics)

In [29]:
# Reference from https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# removing punctuations and unnecessary characters altogether
def sent_to_words(sentences):
    #print(sentences)
    for sentence in sentences:
        #print(sentence[1])
        yield(gensim.utils.simple_preprocess(str(sentences[1]), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(D))

# Step 9
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
# print(trigram_mod[bigram_mod[data_words[0]]])

# Define functions for stopwords, bigrams, trigrams and lemmatization
# Step 10
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#print(download('en'))
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner']) 

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

#Step 11
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[['choose', 'southern', 'state', 'backdrop', 'conservative', 'speech', 'presidential', 'campaign', 'mr', 'gore', 'propose', 'federal', 'spending', 'year', 'help', 'state', 'test', 'treat', 'counsel', 'prisoner', 'parole', 'drug']]


[[('backdrop', 1),
  ('campaign', 1),
  ('choose', 1),
  ('conservative', 1),
  ('counsel', 1),
  ('drug', 1),
  ('federal', 1),
  ('gore', 1),
  ('help', 1),
  ('mr', 1),
  ('parole', 1),
  ('presidential', 1),
  ('prisoner', 1),
  ('propose', 1),
  ('southern', 1),
  ('speech', 1),
  ('spending', 1),
  ('state', 2),
  ('test', 1),
  ('treat', 1),
  ('year', 1)]]

In [30]:
# Step 12: setup LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=NUM_TOPICS, 
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto')

#Step 13
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.048*"presidential" + 0.048*"parole" + 0.048*"treat" + 0.048*"test" + '
  '0.048*"state" + 0.048*"spending" + 0.048*"speech" + 0.048*"southern" + '
  '0.048*"propose" + 0.048*"prisoner"'),
 (1,
  '0.091*"state" + 0.045*"campaign" + 0.045*"spending" + 0.045*"presidential" '
  '+ 0.045*"mr" + 0.045*"help" + 0.045*"speech" + 0.045*"treat" + 0.045*"year" '
  '+ 0.045*"test"'),
 (2,
  '0.048*"spending" + 0.048*"federal" + 0.048*"test" + 0.048*"presidential" + '
  '0.048*"southern" + 0.048*"counsel" + 0.048*"propose" + 0.048*"year" + '
  '0.048*"state" + 0.048*"mr"'),
 (3,
  '0.048*"state" + 0.048*"drug" + 0.048*"backdrop" + 0.048*"propose" + '
  '0.048*"southern" + 0.048*"conservative" + 0.048*"counsel" + 0.048*"choose" '
  '+ 0.048*"spending" + 0.048*"gore"'),
 (4,
  '0.048*"presidential" + 0.048*"parole" + 0.048*"treat" + 0.048*"test" + '
  '0.048*"state" + 0.048*"spending" + 0.048*"speech" + 0.048*"southern" + '
  '0.048*"propose" + 0.048*"prisoner"'),
 (5,
  '0.048*"state" + 0.

In [31]:
# Step 14
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -3.077459966137108

Coherence Score:  0.9999999999999998


# Algorithm Setup

### Setup Iowa Electronic Markets (IEM) Timeseries Data

In [32]:
tsp_df = pd.read_csv("timeSeriesPrices.csv")

# convert $Volume column to float64
tsp_df["$Volume"] = tsp_df["$Volume"].apply(lambda x: x.replace(',', ''))
tsp_df["$Volume"] = tsp_df["$Volume"].astype(float)
tsp_df["$Volume"].loc[tsp_df["$Volume"] == 0] = 0.001 # replace zero values

# setup normalized_tsp_df. each index is a day from 05/01/00 to 10/31/00
gore_prices = tsp_df.loc[tsp_df.Contract == "Dem"]["$Volume"].reset_index(drop=True)
bush_prices = tsp_df.loc[tsp_df.Contract == "Rep"]["$Volume"].reset_index(drop=True)
normalized_tsp_df = gore_prices / (bush_prices + gore_prices)

In [33]:
def sig(pvalue):
    return 1 - pvalue

def get_max_significance_key(gc_res):
    max_key = 1
    max_val = -1
    for key in gc_res.keys():
        if gc_res[key][0]['params_ftest'][0] > max_val:
            max_key = key
            max_val = gc_res[key][0]['params_ftest'][0]
    return max_key

def setup_topic_stream(lda_corpus, timestamps):
    document_topic_coverages = []
    topic_coverage = [0 for i in range(NUM_TOPICS)]
    for i, coverage in enumerate(lda_corpus):
        for j, topic in enumerate(coverage):
            topic_coverage[topic[0]] = topic[1]
        document_topic_coverages.append([timestamps[i]] + topic_coverage)
        topic_coverage = [0 for i in range(NUM_TOPICS)]
    
    # set up dataframe
    fields = ["Date"] + ["Topic"+str(i) for i in range(NUM_TOPICS)]
    topic_stream_df = pd.DataFrame(document_topic_coverages, columns=fields)
    return topic_stream_df

def get_candidate_causal_topics(topic_stream_df, tsp_df, gamma):
    CT = []
    topic_list = ["Topic"+str(i) for i in range(NUM_TOPICS)]
    for i, topic in enumerate(topic_list):
        x = np.array([topic_stream_df[topic].tolist(), normalized_tsp_df.tolist()]).T # TODO: MAY HAVE TO SWITCH THESE
        gc_res = grangercausalitytests(x, 5)
        key = get_max_significance_key(gc_res)
        pvalue = gc_res[key][0]['params_ftest'][1]
        significance = sig(pvalue)
        if significance > gamma:
            CT.append(topic)
    return CT

In [34]:
# step 2
lda_corpus = lda_model[corpus] # setup the topic coverage for each document
topic_stream_df = setup_topic_stream(lda_corpus, T)
topic_stream_df = topic_stream_df.groupby("Date").sum()
topic_stream_df
# CT = get_candidate_causal_topics(topic_stream_df, normalized_tsp_df, 0.5)

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9
Date,,,,,,,,,,
05/03/00,0,76.7555,0,0,0,0,0,0,0,0


In [35]:
# STEP 3
'''
For each candidate topic in CT, apply C to find the mostsignificant causal words among top wordsw∈T. 
Recordthe impact values of these significant words (e.g., word-levelPearson correlations with the time series variable)
'''
def setup_wordcount_stream(timestamps):
    words = [[id2word[id] for id, freq in cp] for cp in corpus[:1]][0]
    df_rows = []
    for i, doc in enumerate(D):
        wordcount_dict = {word: 0  for i, word in enumerate(words)} 
        for word in doc.lower().split(" "):
            if (word.lower() in wordcount_dict):
                wordcount_dict[word] += 1
        df_rows.append([timestamps[i]] + [val for val in wordcount_dict.values()])
    
    # set up dataframe
    fields = ["Date"] + [words[i] for i in range(len(words))]
    wordcount_stream_df = pd.DataFrame(df_rows, columns=fields)
    return wordcount_stream_df


# return array [(word, impact_value)]
def get_significant_causal_topics(wordcount_stream_df, tsp_df, gamma):
    impact_values = []
    words = [[id2word[id] for id, freq in cp] for cp in corpus[:1]][0]
    for i, word in enumerate(words):
        x = np.array([wordcount_stream_df[word].tolist(), tsp_df.tolist()]).T
#         gc_res = grangercausalitytests(x, 5)
#         key = get_max_significance_key(gc_res)
#         pvalue = gc_res[key][0]['params_ftest'][1]
#         significance = sig(pvalue)
#         if significance > gamma:
#             SCT.append(word)
        pvalue = sp.stats.pearsonr(x, tsp_df)
        significance = sig(pvalue)
        if significance > gamma:
            impact_values.append((word, pvalue))
    return impact_values 
    
    
#     CT = []
#     topic_list = ["Topic"+str(i) for i in range(NUM_TOPICS)]
#     for i, topic in enumerate(topic_list):
#         x = np.array([topic_stream_df[topic].tolist(), normalized_tsp_df.tolist()]).T # TODO: MAY HAVE TO SWITCH THESE
#         gc_res = grangercausalitytests(x, 5)
#         key = get_max_significance_key(gc_res)
#         pvalue = gc_res[key][0]['params_ftest'][1]
#         significance = sig(pvalue)
#         if significance > gamma:
#             CT.append(topic)
#     return CT
     
# # One is the external time series (e.g. prices) 
# #and the other is the sum of word counts per time-stamp. You can find more details in section 4.2.2 
# for topic in CT:
#     # apply C
#     break


wordcount_stream_df = setup_wordcount_stream(T)

In [36]:
normalized_tsp_df
# words = [[id2word[id] for id, freq in cp] for cp in corpus[:1]][0]

0      0.659104
1      0.372295
2      0.645607
3      0.286292
4      0.739999
5      0.242304
6      0.000094
7      0.130042
8      0.363381
9      0.438038
10     0.450395
11     0.743199
12     0.544253
13     0.999332
14     0.497994
15     0.587027
16     0.591153
17     0.588401
18     0.359324
19     0.986226
20     0.357826
21     0.616880
22     0.762738
23     0.745276
24     0.031323
25     0.716387
26     0.323849
27     0.998128
28     0.999923
29     0.033904
         ...   
152    0.633310
153    0.399007
154    0.655756
155    0.659162
156    0.408249
157    0.304458
158    0.587465
159    0.645196
160    0.493369
161    0.346190
162    0.599063
163    0.515088
164    0.214619
165    0.389410
166    0.308602
167    0.588639
168    0.859825
169    0.470041
170    0.717222
171    0.490814
172    0.371428
173    0.430711
174    0.519166
175    0.571434
176    0.532200
177    0.484197
178    0.371508
179    0.363610
180    0.542174
181    0.303600
Name: $Volume, Length: 1

# Algorithm Execution

In [37]:
len([id2word.doc2bow(text) for text in texts])

77

In [38]:
data = sm.datasets.macrodata.load_pandas()
data = data.data[["realgdp", "realcons"]].pct_change().dropna()
gc_res = grangercausalitytests(data, 4)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=28.7248 , p=0.0000  , df_denom=198, df_num=1
ssr based chi2 test:   chi2=29.1600 , p=0.0000  , df=1
likelihood ratio test: chi2=27.2295 , p=0.0000  , df=1
parameter F test:         F=28.7248 , p=0.0000  , df_denom=198, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=18.9880 , p=0.0000  , df_denom=195, df_num=2
ssr based chi2 test:   chi2=38.9498 , p=0.0000  , df=2
likelihood ratio test: chi2=35.5873 , p=0.0000  , df=2
parameter F test:         F=18.9880 , p=0.0000  , df_denom=195, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=13.5015 , p=0.0000  , df_denom=192, df_num=3
ssr based chi2 test:   chi2=41.9812 , p=0.0000  , df=3
likelihood ratio test: chi2=38.0914 , p=0.0000  , df=3
parameter F test:         F=13.5015 , p=0.0000  , df_denom=192, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=10.9646 , p=0.0000  

In [39]:
# Step 4
def generate_prior(significance):
    
    #Define a prior on the topic model parameters using significant terms and their impact values. 
    #Step 4a
    pos_impact = [sig - .90 for sig in significance]
    neg_impact = [-sig - .90 for sig in significance]
    for i, sig in enumerate(pos_impact):
        if sig < 0:
            pos_impact[i] = 0
            
    for i, sig in enumerate(neg_impact):
        if sig < 0:
            neg_impact[i] = 0
        
    pos_impact = numpy.array(pos_impact)
    neg_impact = numpy.array(neg_impact)
    
    prior = []
    for i in range(len(pos_impact)):
        count_pos = np.count_nonzero(positive_sigs)
        count_neg = np.count_nonzero(negative_sigs)
        sum_impact = count_pos + count_neg
        if sum_impact != 0:
            prob = count_pos / sum_impact
            #step 4b
            if (prob > 0.9):
                prior.append(pos_impact / np.sum(pos_impact))
            elif (prob < 0.1):
                prior.append(neg_impact / np.sum(neg_impact))
            else:
                prior.append(pos_impact / np.sum(pos_impact))
                prior.append(neg_impact / np.sum(neg_impact))                
    
    if prior != None:
        return numpy.array(prior)

    return None

In [40]:
#step 6
iteration = 0
iteration_num = 10

while iteration < iteration_num:
    iteration += 1
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=NUM_TOPICS, 
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto')
    
    sigTopics = lda_model.get_topics()
    topic_prior = generate_prior(sigTopics)
    doc_lda = lda_model[corpus]
    
pprint(lda_model.print_topics())
    
    

[(0,
  '0.048*"state" + 0.048*"choose" + 0.048*"presidential" + 0.048*"counsel" + '
  '0.048*"mr" + 0.048*"drug" + 0.048*"spending" + 0.048*"southern" + '
  '0.048*"prisoner" + 0.048*"help"'),
 (1,
  '0.050*"state" + 0.048*"choose" + 0.048*"speech" + 0.048*"help" + '
  '0.048*"federal" + 0.048*"spending" + 0.048*"presidential" + '
  '0.048*"backdrop" + 0.048*"southern" + 0.048*"campaign"'),
 (2,
  '0.049*"state" + 0.048*"parole" + 0.048*"choose" + 0.048*"treat" + '
  '0.048*"help" + 0.048*"propose" + 0.048*"spending" + 0.048*"year" + '
  '0.048*"presidential" + 0.048*"test"'),
 (3,
  '0.087*"state" + 0.047*"presidential" + 0.046*"gore" + 0.046*"help" + '
  '0.046*"conservative" + 0.046*"year" + 0.046*"drug" + 0.046*"prisoner" + '
  '0.046*"choose" + 0.046*"parole"'),
 (4,
  '0.054*"state" + 0.048*"propose" + 0.048*"parole" + 0.048*"mr" + '
  '0.048*"choose" + 0.048*"campaign" + 0.048*"gore" + 0.048*"treat" + '
  '0.048*"test" + 0.047*"prisoner"'),
 (5,
  '0.059*"state" + 0.049*"test" +